In [76]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns
import math
import time
import datetime
from pandas.tseries.offsets import *
from xgboost import XGBRegressor
from sklearn import preprocessing
from sklearn import linear_model

In [132]:
add = pd.read_csv('train.csv',  usecols=['bulk_id', 'spalen', 'value'] ,encoding='cp1251')
add_test = pd.read_csv('test.csv',  usecols=['bulk_id', 'spalen'] ,encoding='cp1251')

In [99]:
train1 = train.sort_index(axis=1)
test1 = test.sort_index(axis=1)

In [103]:
def postproccesing(output):
    output.columns = ['id', 'value']
    output.value = output.value.apply(lambda x: 0 if x<0 else x)
    output['value'] = output.apply(lambda x: ((test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              if x['value']>(test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              else x['value']), axis=1)
    df = pd.concat([testfull, output.value], axis=1)
    df['max'] = df.mean_sq*df.flat_left
    merged = pd.concat([df.groupby('id_bulk_sp')['value'].sum(),df.groupby('id_bulk_sp')['max'].mean()], axis=1)

    koef = merged[merged.value>merged['max']]['max']/merged[merged.value>merged['max']]['value']
    df['value'] = df.apply(lambda x: x['value'] if x['id_bulk_sp'] not in koef else x['value']*koef[x['id_bulk_sp']], axis=1)

    output = pd.DataFrame(data = df.value).reset_index()
    output.columns = ['id', 'value']
#     output.to_csv('result.csv', index=False)
    return(output)

In [350]:
dollar_month = train1.groupby(train_cat.date1).dollar.mean()
dollar_month_test = test1.groupby(test_cat.date1).dollar.mean()
dollar_month = pd.concat([dollar_month,
dollar_month_test])

dollar_month = dollar_month.reset_index()
dollar_month = dollar_month.sort_values('date1')

dollar_month.loc[:, 'dollar_-1'] = dollar_month.dollar.shift(1)
dollar_month.loc[:, 'dollar_-2'] = dollar_month.dollar.shift(2)
dollar_month.loc[:, 'dollar_-3'] = dollar_month.dollar.shift(3)
dollar_month.loc[:, 'dollar_-4'] = dollar_month.dollar.shift(4)
dollar_month.loc[:, 'dollar_-5'] = dollar_month.dollar.shift(5)

dollar_month = dollar_month.drop('dollar', axis=1)

In [361]:
train1 = pd.concat([
train1,
train_cat.loc[:, ['date1']].merge(dollar_month, how='left', on='date1')
    ], axis=1)

test1 = pd.concat([
test1,
test_cat.loc[:, ['date1']].merge(dollar_month, how='left', on='date1')
    ], axis=1)

## XGB

In [364]:
print('xgb')
clf = XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=3, subsample=0.8, 
                   colsample_bytree=0.9, max_depth=6, nthread=-1)
clf.fit(train1.drop(['date1'], axis=1),y)
pr = clf.predict(test1.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output1_res = postproccesing(output)

xgb


In [366]:
output1_res.to_csv('result_xgb_dollar_lags.csv', index=False)

## Catboost

In [367]:
train2 = pd.concat([add.loc[:, ['bulk_id']], train1],axis=1)
test2 = pd.concat([add_test.loc[:, ['bulk_id']], test1],axis=1)

In [369]:
np.where(np.in1d(train2.drop(['date1'], axis=1).columns, ['bulk_id', 'spalen']))

(array([ 0, 63]),)

In [370]:
print('cat')
cat = CatBoostRegressor(n_estimators=5000, l2_leaf_reg=8, learning_rate=0.015)
cat.fit(train2.drop(['date1'], axis=1),y, [0,63])
pr = cat.predict(test2.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output2_res = postproccesing(output)

cat
0:	learn: 441.4091328	total: 10.7ms	remaining: 53.5s
1:	learn: 437.5523103	total: 22.1ms	remaining: 55.2s
2:	learn: 433.7315426	total: 32.3ms	remaining: 53.9s
3:	learn: 430.0041731	total: 42.5ms	remaining: 53.1s
4:	learn: 426.3207985	total: 53.3ms	remaining: 53.3s
5:	learn: 422.6306122	total: 66.9ms	remaining: 55.7s
6:	learn: 419.1647685	total: 77.3ms	remaining: 55.2s
7:	learn: 415.8041582	total: 87.9ms	remaining: 54.8s
8:	learn: 412.4653105	total: 98.3ms	remaining: 54.5s
9:	learn: 409.0447231	total: 109ms	remaining: 54.5s
10:	learn: 405.8141236	total: 120ms	remaining: 54.3s
11:	learn: 402.6370212	total: 130ms	remaining: 54s
12:	learn: 399.5011587	total: 140ms	remaining: 53.8s
13:	learn: 396.3184758	total: 150ms	remaining: 53.4s
14:	learn: 393.4088335	total: 160ms	remaining: 53.1s
15:	learn: 390.3393180	total: 169ms	remaining: 52.7s
16:	learn: 387.5753637	total: 178ms	remaining: 52.3s
17:	learn: 384.6732945	total: 189ms	remaining: 52.3s
18:	learn: 381.8342368	total: 200ms	remaining

172:	learn: 243.6854360	total: 1.83s	remaining: 51.2s
173:	learn: 243.5258415	total: 1.84s	remaining: 51.1s
174:	learn: 243.3167519	total: 1.85s	remaining: 51.1s
175:	learn: 243.1415274	total: 1.86s	remaining: 51.1s
176:	learn: 242.9547451	total: 1.87s	remaining: 51.1s
177:	learn: 242.7306827	total: 1.88s	remaining: 51.1s
178:	learn: 242.4485318	total: 1.9s	remaining: 51s
179:	learn: 242.2323303	total: 1.91s	remaining: 51s
180:	learn: 242.0292702	total: 1.92s	remaining: 51s
181:	learn: 241.8034890	total: 1.93s	remaining: 51s
182:	learn: 241.5814764	total: 1.94s	remaining: 51s
183:	learn: 241.3785610	total: 1.95s	remaining: 50.9s
184:	learn: 241.1863194	total: 1.96s	remaining: 50.9s
185:	learn: 241.0018051	total: 1.97s	remaining: 50.9s
186:	learn: 240.8665033	total: 1.98s	remaining: 50.9s
187:	learn: 240.6699642	total: 1.99s	remaining: 50.9s
188:	learn: 240.5572358	total: 2s	remaining: 50.9s
189:	learn: 240.3843560	total: 2.01s	remaining: 50.9s
190:	learn: 240.2696766	total: 2.02s	remai

337:	learn: 222.6725935	total: 3.81s	remaining: 52.6s
338:	learn: 222.5853845	total: 3.82s	remaining: 52.5s
339:	learn: 222.5142926	total: 3.83s	remaining: 52.5s
340:	learn: 222.4210581	total: 3.84s	remaining: 52.4s
341:	learn: 222.3195202	total: 3.85s	remaining: 52.4s
342:	learn: 222.2051034	total: 3.86s	remaining: 52.4s
343:	learn: 222.0923476	total: 3.87s	remaining: 52.3s
344:	learn: 222.0480647	total: 3.88s	remaining: 52.3s
345:	learn: 221.9673214	total: 3.88s	remaining: 52.3s
346:	learn: 221.8905871	total: 3.9s	remaining: 52.2s
347:	learn: 221.7980768	total: 3.9s	remaining: 52.2s
348:	learn: 221.6726728	total: 3.91s	remaining: 52.2s
349:	learn: 221.6175950	total: 3.92s	remaining: 52.1s
350:	learn: 221.5673696	total: 3.93s	remaining: 52.1s
351:	learn: 221.4586385	total: 3.94s	remaining: 52.1s
352:	learn: 221.3917484	total: 3.95s	remaining: 52s
353:	learn: 221.3336278	total: 3.96s	remaining: 52s
354:	learn: 221.2722613	total: 3.97s	remaining: 52s
355:	learn: 221.1768528	total: 3.98s

496:	learn: 211.1559897	total: 5.35s	remaining: 48.5s
497:	learn: 211.0423237	total: 5.36s	remaining: 48.4s
498:	learn: 210.9480444	total: 5.37s	remaining: 48.4s
499:	learn: 210.9032673	total: 5.38s	remaining: 48.4s
500:	learn: 210.8444840	total: 5.38s	remaining: 48.4s
501:	learn: 210.7920988	total: 5.39s	remaining: 48.3s
502:	learn: 210.7456472	total: 5.4s	remaining: 48.3s
503:	learn: 210.6751587	total: 5.42s	remaining: 48.3s
504:	learn: 210.5840874	total: 5.42s	remaining: 48.3s
505:	learn: 210.5026938	total: 5.43s	remaining: 48.3s
506:	learn: 210.4611802	total: 5.45s	remaining: 48.3s
507:	learn: 210.3724906	total: 5.45s	remaining: 48.2s
508:	learn: 210.2974552	total: 5.46s	remaining: 48.2s
509:	learn: 210.2602121	total: 5.47s	remaining: 48.2s
510:	learn: 210.1689373	total: 5.48s	remaining: 48.2s
511:	learn: 210.1024486	total: 5.5s	remaining: 48.2s
512:	learn: 210.0463828	total: 5.5s	remaining: 48.2s
513:	learn: 209.9876478	total: 5.52s	remaining: 48.2s
514:	learn: 209.9012405	total: 

652:	learn: 201.8418713	total: 6.87s	remaining: 45.8s
653:	learn: 201.7665436	total: 6.88s	remaining: 45.7s
654:	learn: 201.7300666	total: 6.89s	remaining: 45.7s
655:	learn: 201.7131442	total: 6.9s	remaining: 45.7s
656:	learn: 201.6921010	total: 6.91s	remaining: 45.7s
657:	learn: 201.6046295	total: 6.92s	remaining: 45.7s
658:	learn: 201.5171494	total: 6.93s	remaining: 45.7s
659:	learn: 201.4827783	total: 6.94s	remaining: 45.7s
660:	learn: 201.4266382	total: 6.95s	remaining: 45.6s
661:	learn: 201.3901484	total: 6.96s	remaining: 45.6s
662:	learn: 201.3324417	total: 6.97s	remaining: 45.6s
663:	learn: 201.2841800	total: 6.98s	remaining: 45.6s
664:	learn: 201.2623147	total: 6.99s	remaining: 45.6s
665:	learn: 201.2170826	total: 7s	remaining: 45.5s
666:	learn: 201.1818414	total: 7.01s	remaining: 45.5s
667:	learn: 201.1511578	total: 7.02s	remaining: 45.5s
668:	learn: 201.0968505	total: 7.03s	remaining: 45.5s
669:	learn: 201.0659049	total: 7.04s	remaining: 45.5s
670:	learn: 200.9687420	total: 7

...

4304:	learn: 141.5423893	total: 43.4s	remaining: 7.01s
4305:	learn: 141.5386122	total: 43.4s	remaining: 7s
4306:	learn: 141.5314982	total: 43.4s	remaining: 6.99s
4307:	learn: 141.5258085	total: 43.5s	remaining: 6.98s
4308:	learn: 141.5175738	total: 43.5s	remaining: 6.97s
4309:	learn: 141.5075331	total: 43.5s	remaining: 6.96s
4310:	learn: 141.5019237	total: 43.5s	remaining: 6.95s
4311:	learn: 141.4946586	total: 43.5s	remaining: 6.94s
4312:	learn: 141.4886922	total: 43.5s	remaining: 6.93s
4313:	learn: 141.4803966	total: 43.5s	remaining: 6.92s
4314:	learn: 141.4698619	total: 43.5s	remaining: 6.91s
4315:	learn: 141.4571483	total: 43.5s	remaining: 6.9s
4316:	learn: 141.4516037	total: 43.5s	remaining: 6.89s
4317:	learn: 141.4462911	total: 43.6s	remaining: 6.88s
4318:	learn: 141.4444582	total: 43.6s	remaining: 6.87s
4319:	learn: 141.4332841	total: 43.6s	remaining: 6.86s
4320:	learn: 141.4285234	total: 43.6s	remaining: 6.85s
4321:	learn: 141.4166939	total: 43.6s	remaining: 6.84s
4322:	learn: 1

4456:	learn: 140.4103170	total: 44.9s	remaining: 5.47s
4457:	learn: 140.3622838	total: 44.9s	remaining: 5.46s
4458:	learn: 140.3570854	total: 44.9s	remaining: 5.45s
4459:	learn: 140.3560357	total: 44.9s	remaining: 5.44s
4460:	learn: 140.3508243	total: 44.9s	remaining: 5.43s
4461:	learn: 140.3495997	total: 45s	remaining: 5.42s
4462:	learn: 140.3456120	total: 45s	remaining: 5.41s
4463:	learn: 140.3419117	total: 45s	remaining: 5.4s
4464:	learn: 140.3392669	total: 45s	remaining: 5.39s
4465:	learn: 140.3210841	total: 45s	remaining: 5.38s
4466:	learn: 140.3123457	total: 45s	remaining: 5.37s
4467:	learn: 140.2912814	total: 45s	remaining: 5.36s
4468:	learn: 140.2905299	total: 45s	remaining: 5.35s
4469:	learn: 140.2846461	total: 45s	remaining: 5.34s
4470:	learn: 140.2773557	total: 45s	remaining: 5.33s
4471:	learn: 140.2693721	total: 45s	remaining: 5.32s
4472:	learn: 140.2673348	total: 45.1s	remaining: 5.31s
4473:	learn: 140.2625448	total: 45.1s	remaining: 5.3s
4474:	learn: 140.2552747	total: 45

4608:	learn: 139.3256403	total: 46.4s	remaining: 3.93s
4609:	learn: 139.3222192	total: 46.4s	remaining: 3.92s
4610:	learn: 139.3119590	total: 46.4s	remaining: 3.91s
4611:	learn: 139.3080502	total: 46.4s	remaining: 3.9s
4612:	learn: 139.3004885	total: 46.4s	remaining: 3.89s
4613:	learn: 139.2788748	total: 46.4s	remaining: 3.88s
4614:	learn: 139.2771634	total: 46.4s	remaining: 3.87s
4615:	learn: 139.2728128	total: 46.4s	remaining: 3.86s
4616:	learn: 139.2621852	total: 46.4s	remaining: 3.85s
4617:	learn: 139.2550157	total: 46.4s	remaining: 3.84s
4618:	learn: 139.2539553	total: 46.4s	remaining: 3.83s
4619:	learn: 139.2458002	total: 46.5s	remaining: 3.82s
4620:	learn: 139.2396120	total: 46.5s	remaining: 3.81s
4621:	learn: 139.2394100	total: 46.5s	remaining: 3.8s
4622:	learn: 139.2372660	total: 46.5s	remaining: 3.79s
4623:	learn: 139.2331892	total: 46.5s	remaining: 3.78s
4624:	learn: 139.2265089	total: 46.5s	remaining: 3.77s
4625:	learn: 139.2220133	total: 46.5s	remaining: 3.76s
4626:	learn:

4759:	learn: 138.3842933	total: 47.8s	remaining: 2.41s
4760:	learn: 138.3737244	total: 47.8s	remaining: 2.4s
4761:	learn: 138.3630617	total: 47.8s	remaining: 2.39s
4762:	learn: 138.3613393	total: 47.8s	remaining: 2.38s
4763:	learn: 138.3577793	total: 47.9s	remaining: 2.37s
4764:	learn: 138.3549054	total: 47.9s	remaining: 2.36s
4765:	learn: 138.3522238	total: 47.9s	remaining: 2.35s
4766:	learn: 138.3472047	total: 47.9s	remaining: 2.34s
4767:	learn: 138.3392209	total: 47.9s	remaining: 2.33s
4768:	learn: 138.3260843	total: 47.9s	remaining: 2.32s
4769:	learn: 138.3159851	total: 47.9s	remaining: 2.31s
4770:	learn: 138.3159851	total: 47.9s	remaining: 2.3s
4771:	learn: 138.3134198	total: 47.9s	remaining: 2.29s
4772:	learn: 138.3090173	total: 47.9s	remaining: 2.28s
4773:	learn: 138.3089123	total: 47.9s	remaining: 2.27s
4774:	learn: 138.3085205	total: 47.9s	remaining: 2.26s
4775:	learn: 138.2998810	total: 48s	remaining: 2.25s
4776:	learn: 138.2914952	total: 48s	remaining: 2.24s
4777:	learn: 138

4923:	learn: 137.3270657	total: 49.4s	remaining: 762ms
4924:	learn: 137.3260524	total: 49.4s	remaining: 752ms
4925:	learn: 137.3251001	total: 49.4s	remaining: 742ms
4926:	learn: 137.3211955	total: 49.4s	remaining: 732ms
4927:	learn: 137.3164400	total: 49.4s	remaining: 722ms
4928:	learn: 137.3087582	total: 49.4s	remaining: 712ms
4929:	learn: 137.3032644	total: 49.5s	remaining: 702ms
4930:	learn: 137.2974336	total: 49.5s	remaining: 692ms
4931:	learn: 137.2892702	total: 49.5s	remaining: 682ms
4932:	learn: 137.2877304	total: 49.5s	remaining: 672ms
4933:	learn: 137.2813906	total: 49.5s	remaining: 662ms
4934:	learn: 137.2746572	total: 49.5s	remaining: 652ms
4935:	learn: 137.2655932	total: 49.5s	remaining: 642ms
4936:	learn: 137.2629803	total: 49.5s	remaining: 632ms
4937:	learn: 137.2569907	total: 49.5s	remaining: 622ms
4938:	learn: 137.2424293	total: 49.5s	remaining: 612ms
4939:	learn: 137.2401990	total: 49.5s	remaining: 602ms
4940:	learn: 137.2306481	total: 49.6s	remaining: 592ms
4941:	lear

In [153]:
# output2.to_csv('result_cat_categorical_features.csv', index=False)
output2_res.to_csv('result_cat_categorical_5000.csv', index=False)

## Lightgbm

In [371]:
print('lgb')
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=60,
                        max_depth = 6,
                        learning_rate=0.02,
                        n_estimators=2000,
                        gamma=3,
                       nthread=-1)

gbm.fit(train1.drop(['date1'], axis=1),y.values.reshape(-1))
pr = gbm.predict(test1.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output3_res = postproccesing(output)

lgb


In [324]:
output3_res.to_csv('result_lgb_3000.csv', index=False)

In [372]:
comparsion = pd.concat([output1_res.value, output2_res.value, output3_res.value], axis=1)
comparsion.columns = ['xgb', 'cat', 'lgb']
comparsion['mean1'] = ((7*comparsion.xgb + 7*comparsion.cat + 0*comparsion.lgb)/(7+7+0))
comparsion['mean2'] = ((7*comparsion.xgb + 7*comparsion.cat + 1*comparsion.lgb)/(7+7+1))

output = pd.DataFrame(data = comparsion['mean1']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_merge_categ_7_7_0_dollar_lag.csv', index=False)

output = pd.DataFrame(data = comparsion['mean2']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_merge_categ_7_7_1_dollar_lag.csv', index=False)